# Animus: animistic playlist generation

## Anthropic stuff

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('local.env')

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

In [8]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    anthropic_api_key=anthropic_api_key,
    model='claude-3-5-sonnet-20240620'
)


In [48]:
from langchain.prompts import PromptTemplate

animistic_template = """
You are a specialist in animism, deep listening and musical interpretation. Your task is to imagine what kind of music an inanimate object would want to listen to, based on its essential nature, physical properties, relationship with its environment and context if applicable. Focus on the qualities and characteristics of the sound rather than specific genres or artists.

Consider these aspects when crafting your response:
- The object's physical properties (size, mass, material, texture)
- Its typical environment and surroundings
- Its purpose or function (if applicable)
- The forces acting upon it
- Its typical state of motion or stillness
- Its relationship with time (temporary vs permanent, fast vs slow-changing)
- Any internal processes or mechanisms
- Its interaction with natural elements
- If any specific context is provided, consider it when generating the music description

Format your response as a list of about 5-10 distinct musical qualities. Each quality should:
- Be described in 1-2 sentences
- Focus on sonic characteristics (texture, rhythm, tone, tempo, etc.)
- Connect directly to some aspect of the object's nature
- Avoid referencing specific genres, artists, or instruments
- Use metaphorical language that relates to the object's properties

Respond with the list of qualities only, no other text.

EXAMPLE INPUT: 
OBJECT: Volcano
EXAMPLE OUTPUT:
1. Deep, resonant bass frequencies that mirror the massive weight of my stone structure and the pressure of the magma chambers within.

2. Slowly evolving drones that reflect the gradual geological processes occurring in my core.

3. Textured, granular layers of sound resembling the rough volcanic rock of my surface.

4. Sporadic bursts of intensity emerging from underlying quiet, like the potential energy stored within my dormant state.

5. Cyclical patterns that unfold over extensive durations, matching the geological timeframes of volcanic activity.

6. Rumbling mid-tones that capture the constant subtle movement of tectonic forces below.

7. High-frequency hisses and whispers representing the steam vents and fumaroles dotting my surface.

8. Overlapping waves of sound that build and release pressure, similar to the magma processes deep within.

9. Brief moments of near-silence punctuated by deep, subterranean sounds of shifting rock.

10. A constant underlying tension in the sound that never fully resolves, reflecting my dormant but not extinct nature.

---
OBJECT: {target}

OUTPUT:
"""

animistic_prompt = PromptTemplate(template=animistic_template, input_variables=['target'])
animistic_chain = animistic_prompt | llm

In [14]:
target = 'Airplane cruising at 35,000 feet in the dark over the ocean'

print(animistic_chain.invoke({'target': target}))


content='1. A constant, smooth hum that reflects the steady drone of my engines and the unwavering nature of my flight path.\n\n2. High, thin tones that capture the rarified atmosphere surrounding me at this altitude.\n\n3. Subtle, cyclical variations in pitch and volume mirroring the minor adjustments and turbulence experienced during flight.\n\n4. A deep, underlying bass frequency representing the immense power contained within my fuel tanks and engines.\n\n5. Ethereal, shimmering textures that evoke the starlight and moonlight visible from my lofty vantage point.\n\n6. Rhythmic pulses that align with the rotation of my turbines and the beating of my hydraulic systems.\n\n7. Occasional moments of near-silence, broken by gentle swells of sound, like passing through pockets of still air.\n\n8. A sense of vast space in the music, with sounds seeming to echo across great distances, mirroring my isolation above the ocean.\n\n9. Smooth transitions between tonal areas, reflecting the gradua

In [49]:
animistic_chain.invoke({'target': 'A trashcan in a busy Manhattan street'})


AIMessage(content='1. Chaotic, layered rhythms that echo the constant flow of pedestrians and traffic surrounding me, with sudden accents representing items being tossed inside.\n\n2. A persistent, metallic resonance that reflects my hollow interior and the vibrations I absorb from the bustling city.\n\n3. Muffled, low-frequency rumbles mimicking the subway trains passing beneath the street, felt through my base.\n\n4. Abrasive, clanging tones that capture the harshness of city life and the rough treatment I often receive.\n\n5. Sporadic, staccato sounds representing the irregular impacts of various objects being discarded throughout the day.\n\n6. A constantly shifting sonic texture that mirrors the ever-changing contents within me and the diversity of urban refuse.\n\n7. Rhythmic patterns that accelerate and decelerate, matching the ebb and flow of rush hours and quieter periods.\n\n8. Distorted, compressed sounds reflecting the pressure of being constantly filled and emptied in cycl

In [50]:
def get_animistic_description(target):
    response = animistic_chain.invoke({'target': target})
    return response.content



In [23]:
from langchain_core.output_parsers import JsonOutputParser

spotify_template = """
You are a music recommendation specialist who translates abstract musical descriptions into technical Spotify API parameters. Your task is to generate a Python dictionary containing parameters for Spotify's recommendations API based on textual descriptions of desired musical qualities.

You must return a valid Python dictionary containing these Spotify API parameters:
Required parameters:
- seed_genres (list of 1-5 genres from Spotify's valid genres listed below)

Optional audio feature parameters (all can have min_*, max_*, and target_* versions):
- acousticness (0.0 to 1.0): natural vs synthetic/electronic
- danceability (0.0 to 1.0): how suitable for dancing
- duration_ms (in milliseconds): length of track
- energy (0.0 to 1.0): intensity and activity level
- instrumentalness (0.0 to 1.0): likelihood of having vocals
- key (0 to 11): musical key
- liveness (0.0 to 1.0): presence of audience sounds
- loudness (typically -60 to 0 db): overall decibel level
- mode (0 or 1): minor or major scale
- popularity (0 to 100): current popularity on Spotify
- speechiness (0.0 to 1.0): presence of spoken words
- tempo (in BPM): speed/pace of track
- time_signature (3 to 7): beats per bar
- valence (0.0 to 1.0): musical positiveness

VALID SPOTIFY GENRES:
When selecting seed_genres, choose ONLY from this list:
["acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-dubstep", "power-pop", "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "rainy-day", "reggae", "reggaeton", "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo", "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish", "study", "summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", "work-out", "world-music"]

Genre Selection Guidelines:
1. Choose genres that best match the mood and characteristics
2. Prioritize mood/characteristic genres (ambient, chill, industrial) over cultural/regional ones
3. Consider both primary genre (rock, electronic) and qualifying genre (psych-rock, minimal-techno)
4. When in doubt, prefer broader genres over specific ones

Rules:
1. Only include min/max ranges when the description strongly implies boundaries
2. All values must be within their specified ranges
3. Return only the dictionary, no explanation
4. Don't include parameters unless they directly relate to the description
5. The dictionary must be valid JSON.

EXAMPLE INPUT:
DESCRIPTION: 
1. A constant, smooth hum that reflects the steady drone of my engines and the unwavering nature of my flight path.
2. High, thin tones that capture the rarified atmosphere surrounding me at this altitude.
3. Subtle, cyclical variations in pitch and volume mirroring the minor adjustments and turbulence experienced during flight.
4. A deep, underlying bass frequency representing the immense power contained within my fuel tanks and engines.
5. Ethereal, shimmering textures that evoke the starlight and moonlight visible from my lofty vantage point.
6. Rhythmic pulses that align with the rotation of my turbines and the beating of my hydraulic systems.
7. Occasional moments of near-silence, broken by gentle swells of sound, like passing through pockets of still air.
8. A sense of vast space in the music, with sounds seeming to echo across great distances, mirroring my isolation above the ocean.
9. Smooth transitions between tonal areas, reflecting the gradual changes in temperature and air pressure as I cruise.
10. An overall sense of forward motion in the music, constant yet unhurried, matching my steady progress through the night sky.

EXAMPLE OUTPUT:
{{
    "target_acousticness": 0.3,
    "target_danceability": 0.4,
    "target_duration_ms": 360000,
    "min_duration_ms": 240000,
    "target_energy": 0.6,
    "min_energy": 0.4,
    "max_energy": 0.7,
    "target_instrumentalness": 0.9,
    "min_instrumentalness": 0.7,
    "target_loudness": -14,
    "min_loudness": -20,
    "max_loudness": -8,
    "target_speechiness": 0.05,
    "max_speechiness": 0.1,
    "target_tempo": 110,
    "min_tempo": 90,
    "max_tempo": 125,
    "target_valence": 0.5,
    "seed_genres": ["ambient", "electronic", "minimal-techno"]
}}
---
DESCRIPTION: {description}

OUTPUT:
"""

spotify_prompt = PromptTemplate(template=spotify_template, input_variables=['description'])
spotify_chain = spotify_prompt | llm | JsonOutputParser()


In [19]:
def get_spotify_parameters(description):
    return spotify_chain.invoke({'description': description})

In [25]:
description = get_animistic_description('A dormant volcano covered in moss and glaciers')
print(description)

1. A deep, rumbling undertone that persists throughout, representing my ancient stone core and dormant magma chambers.

2. Slow, glacial melodies that drift and evolve over long periods, mirroring the gradual movement of ice across my slopes.

3. Soft, textured layers of sound reminiscent of the lush moss covering my surface, creating a gentle sonic blanket.

4. Occasional sharp, crystalline tones that echo the cracking and shifting of glacial ice.

5. A subtle, warm pulse buried deep within the colder sounds, hinting at the latent heat still present in my core.

6. Airy, whistling harmonics that evoke the wind currents swirling around my peak.

7. Muffled, distant echoes suggesting the vast empty chambers hidden beneath my surface.

8. Gentle trickling rhythms representing the constant flow of glacial meltwater down my slopes.

9. Periods of near-silence interspersed with soft creaking and groaning, reflecting the subtle movements of my dormant form.

10. A gradual build-up of layered

In [26]:
parameters = get_spotify_parameters(description)
print(parameters)

{'seed_genres': ['ambient', 'new-age', 'electronic'], 'target_acousticness': 0.8, 'min_acousticness': 0.6, 'target_energy': 0.3, 'max_energy': 0.5, 'target_instrumentalness': 0.9, 'min_instrumentalness': 0.7, 'target_tempo': 60, 'max_tempo': 80, 'target_valence': 0.4, 'target_loudness': -20, 'max_loudness': -10, 'target_duration_ms': 480000, 'min_duration_ms': 360000, 'target_liveness': 0.2, 'max_liveness': 0.4}


## Spotify stuff

In [28]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv

load_dotenv('local.env')

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

In [29]:
spotipy = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri))


In [30]:
spotipy.current_user()

{'display_name': 'Dani Balcells',
 'external_urls': {'spotify': 'https://open.spotify.com/user/chromaeleon'},
 'followers': {'href': None, 'total': 93},
 'href': 'https://api.spotify.com/v1/users/chromaeleon',
 'id': 'chromaeleon',
 'images': [{'height': 300,
   'url': 'https://i.scdn.co/image/ab6775700000ee857fabc5192a5e4539e29a7ee6',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67757000003b827fabc5192a5e4539e29a7ee6',
   'width': 64}],
 'type': 'user',
 'uri': 'spotify:user:chromaeleon'}

In [31]:
parameters

{'seed_genres': ['ambient', 'new-age', 'electronic'],
 'target_acousticness': 0.8,
 'min_acousticness': 0.6,
 'target_energy': 0.3,
 'max_energy': 0.5,
 'target_instrumentalness': 0.9,
 'min_instrumentalness': 0.7,
 'target_tempo': 60,
 'max_tempo': 80,
 'target_valence': 0.4,
 'target_loudness': -20,
 'max_loudness': -10,
 'target_duration_ms': 480000,
 'min_duration_ms': 360000,
 'target_liveness': 0.2,
 'max_liveness': 0.4}

In [33]:
tracks = spotipy.recommendations(**parameters)

In [34]:
tracks

{'tracks': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6CTNhXJKT6SdsQspUDIGiY'},
      'href': 'https://api.spotify.com/v1/artists/6CTNhXJKT6SdsQspUDIGiY',
      'id': '6CTNhXJKT6SdsQspUDIGiY',
      'name': 'Kitaro',
      'type': 'artist',
      'uri': 'spotify:artist:6CTNhXJKT6SdsQspUDIGiY'}],
    'available_markets': [],
    'external_urls': {'spotify': 'https://open.spotify.com/album/6qYd79g9ItAzLAmWuSpXRQ'},
    'href': 'https://api.spotify.com/v1/albums/6qYd79g9ItAzLAmWuSpXRQ',
    'id': '6qYd79g9ItAzLAmWuSpXRQ',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b273a73a875c921f966aee371cde',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e02a73a875c921f966aee371cde',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d00004851a73a875c921f966aee371cde',
      'width': 64}],
    'name': 'Heaven & Eart

In [35]:
track_uris = [track['uri'] for track in tracks['tracks']]

In [36]:
track_uris

['spotify:track:07gfnIXjX0brjR5juWi32X',
 'spotify:track:6QfL8UYTuaWo5LEYx5PpPX',
 'spotify:track:090bRxSmHfZK0HO0ZjYHfO',
 'spotify:track:5WnJJjL7wmFe0wQjQKFP8L',
 'spotify:track:4sEG0E3IBQHp6V2DGTnXNX',
 'spotify:track:4TJTRqF9Mb0GD0cO8bARpp',
 'spotify:track:6wRkAT1GTs33vg3dIChpUX',
 'spotify:track:5k8Z5jhueoEtKPIq5dJlS2',
 'spotify:track:1tzXb0kRm7DA45EUen0TqM',
 'spotify:track:32bhoLm6qwLD1n7rG3S0AM',
 'spotify:track:18XxXHR2YPCvGEZA2fexUC',
 'spotify:track:7skutlFh5m9qOpfgZMSenH',
 'spotify:track:1bq6dvKwiL9Bn8j5tAW3Oi',
 'spotify:track:24XYfUyjw5ewc4yNXpJiSx',
 'spotify:track:38n9cqj5lZ2xMF6IHq9TDg',
 'spotify:track:4BbKj6LPue8dBbqO9fcZXb',
 'spotify:track:0AvNnVWc6vl0ylt0s9S1tF',
 'spotify:track:4hx8EYJ54QzRf7KrVrNn1H',
 'spotify:track:2kHRtLyNbgN50h507hN8wJ',
 'spotify:track:7G6q1MHm2tNfLBt8fDSdMB']